In [63]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score


columns = ["node_id"]

for i in range(128):
    columns.append("dim"+str(i+1))

# embedding = np.load('embeddings/netmf_cora.out.id.npy')

X = pd.read_csv('/Users/zak/Documents/GitHub/link-prediction-cluster/embeddings/node2vec/blogcat.emb', delim_whitespace=True, names=columns).sort_values(by=['node_id'], ascending=True)
X = X.set_index("node_id")



In [64]:
X

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim119,dim120,dim121,dim122,dim123,dim124,dim125,dim126,dim127,dim128
node_id,,,,,,,,,,,,,,,,,,,,,
1,-0.198050,-0.240728,0.462760,-0.232700,0.065324,-0.075594,-0.093881,-0.101880,-0.108083,0.494183,...,-0.048254,0.003596,-0.035312,0.048063,0.105010,0.005870,-0.438889,0.135354,-0.514053,-0.061525
2,0.102673,-0.167543,0.260817,0.129341,0.394366,0.360594,-0.363314,0.267022,-0.005618,0.053459,...,0.085022,0.045535,-0.112907,0.203329,0.012162,0.184717,-0.210832,-0.006662,-0.063152,-0.019229
3,0.110811,-0.348646,0.342873,-0.064292,0.344434,0.136997,0.112190,0.359515,-0.142944,0.068730,...,0.069237,0.159887,-0.393348,-0.101912,0.071304,0.511129,-0.497600,-0.056946,-0.284930,0.105744
4,-0.252063,0.222983,-0.337673,-0.094082,0.031304,-0.298245,-0.032197,0.022356,-0.175562,0.178323,...,0.091603,-0.103367,-0.131682,-0.136555,0.122139,0.155184,-0.212920,-0.160658,-0.300816,0.035831
5,0.143286,-0.402149,0.254723,0.342771,0.261119,-0.198173,-0.264486,0.305329,0.021822,0.114990,...,0.066580,-0.050502,-0.063459,-0.131586,-0.194582,-0.061572,0.037032,-0.577453,-0.208204,-0.010503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10308,-0.138194,-0.293497,-0.087638,0.162641,0.435378,-0.207839,0.042428,-0.098633,0.006030,0.144831,...,-0.024634,0.244329,0.108833,-0.029809,-0.087627,0.253320,-0.172919,0.346459,0.033685,-0.004634
10309,-0.024161,0.044911,0.113895,0.094565,0.177552,-0.035867,-0.158615,-0.091266,-0.242965,0.099051,...,0.119736,0.029991,-0.154723,-0.268823,0.221713,0.221449,0.002177,0.171322,0.196801,0.128635
10310,-0.161212,-0.427129,-0.073157,0.292491,0.012751,-0.240063,-0.013032,-0.095586,0.413357,0.142965,...,-0.107128,-0.039588,0.240165,-0.065313,0.030545,0.098529,-0.347210,0.596139,0.119845,0.111296


In [79]:

# sort labels after node_ids
labels = pd.read_csv('/Users/zak/Documents/GitHub/Network-Representation-Learning/data/blogcat/data/group-edges.csv',
                     sep=",", header=None, index_col=0)
y = labels.sort_index()
y.columns=["label"]


In [88]:
y

,label
0,
1,21
2,8
3,6
4,24
5,5
...,...
10309,8
10310,29
10311,37


In [85]:
from collections import defaultdict
classes = defaultdict(list)

In [89]:
for index, row in y.iterrows():
   classes[index].append(row["label"])

In [90]:
classes

defaultdict(list,
            {1: [21],
             2: [8],
             3: [6],
             4: [24],
             5: [5],
             6: [21, 8],
             7: [8],
             8: [4],
             9: [9],
             10: [5],
             11: [11],
             12: [22],
             13: [8],
             14: [8, 39],
             15: [8],
             16: [5],
             17: [24],
             18: [8],
             19: [24],
             20: [21],
             21: [8],
             22: [24],
             23: [8],
             24: [30, 24, 19],
             25: [3],
             26: [8],
             27: [22, 11],
             28: [2, 1],
             29: [8],
             30: [19, 30],
             31: [22, 9, 5],
             32: [1],
             33: [18],
             34: [8],
             35: [8],
             36: [6, 1],
             37: [8, 1, 19, 3],
             38: [4],
             39: [36],
             40: [8],
             41: [19, 24],
             42: [5],
  

In [92]:
d = pd.DataFrame(classes.items(), columns=['node', 'labels'])

In [96]:
d = d.set_index("node")

In [98]:
d

,labels
node,
1,[21]
2,[8]
3,[6]
4,[24]
5,[5]
...,...
10308,[29]
10309,[8]
10310,[29]


In [99]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [100]:
forest = RandomForestClassifier(random_state=1)

In [154]:
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [133]:

X_train, X_test, y_train, y_test = train_test_split(
    X, d, random_state=513, train_size=0.8)

In [134]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [135]:
tr = [i for i in range (40)]

In [136]:
label_encoder.fit(tr)

LabelEncoder()

In [137]:
y_train

,labels
node,
5314,"[13, 17]"
1593,[4]
2360,[32]
6253,[7]
2469,[2]
...,...
5685,[19]
836,[22]
2142,[2]


In [138]:
integer_encoded = y_train["labels"].apply(lambda x:label_encoder.transform(x))

In [141]:
integer_encoded

node
5314    [13, 17]
1593         [4]
2360        [32]
6253         [7]
2469         [2]
          ...   
5685        [19]
836         [22]
2142         [2]
6120        [19]
661         [24]
Name: labels, Length: 8249, dtype: object

In [145]:
df3 = pd.DataFrame(y_train['encoded'].to_list())

In [152]:
df3 = df3.fillna(0)

In [157]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [160]:
df3.shape

(8249, 11)

In [161]:
X_train.shape

(8249, 128)

In [159]:

# classifiers = [('qda', QuadraticDiscriminantAnalysis()), ('lda', LinearDiscriminantAnalysis(
# )), ('rf', RandomForestClassifier(criterion='entropy', n_estimators=250, max_features='sqrt'))]
# model = StackingClassifier(classifiers, cv=10)

model = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

multi_target_forest.fit(X_train, df3)

clf = OneVsRestClassifier(SVC()).fit(X_train, df3)

ValueError: Multioutput target data is not supported with label binarization

In [ ]:
y_pred = clf.predict(X_test)

print('F1-Micro', f1_score(y_test, y_pred, average='micro'))
print('F1-Macro', f1_score(y_test, y_pred, average='macro'))

In [54]:
# sort labels after node_ids
data = pd.read_csv('/Users/zak/Documents/GitHub/Network-Representation-Learning/data/cora/cora.content',
                   sep="\t", header=None, index_col=0)
data = data[1434]
y = data.sort_index()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=513, train_size=0.8)


# classifiers = [('qda', QuadraticDiscriminantAnalysis()), ('lda', LinearDiscriminantAnalysis(
# )), ('rf', RandomForestClassifier(criterion='entropy', n_estimators=250, max_features='sqrt'))]
# model = StackingClassifier(classifiers, cv=10)

model = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('F1-Micro', f1_score(y_test, y_pred, average='micro'))
print('F1-Macro', f1_score(y_test, y_pred, average='macro'))

# print(X)
# print(y)


F1-Micro 0.6476014760147601
F1-Macro 0.621706706765971


In [8]:
y

0
35             Genetic_Algorithms
40             Genetic_Algorithms
114        Reinforcement_Learning
117        Reinforcement_Learning
128        Reinforcement_Learning
                    ...          
1154500                Case_Based
1154520           Neural_Networks
1154524             Rule_Learning
1154525             Rule_Learning
1155073             Rule_Learning
Name: 1434, Length: 2708, dtype: object

In [16]:
cv_scores = cross_val_score(model, X, y, cv=5)

In [17]:
cv_scores

array([0.08487085, 0.02398524, 0.08118081, 0.11829945, 0.15526802])

In [18]:
format(np.mean(cv_scores))

'0.09272087360429981'